In [1]:
import napari
import numpy as np
import logging

from blimp.data import load_example_data
from blimp.log import configure_logging
from blimp.constants import blimp_config
from aicsimageio import AICSImage
from pathlib import Path

configure_logging(verbosity=2)
# ensure that example data is downloaded
example_data_path = load_example_data()
# read correct blimp_config -- created with 0_setup.ipynb
blimp_config.config_fname = "blimp.ini"
print(blimp_config)


07-Feb-23 16:10:37 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


Reading config from /Users/scottberry/.config/blimp/blimp.ini
unpacked_dir = /Users/scottberry/source/blimp/notebooks/_data/raw
archive_path = /Users/scottberry/source/blimp/notebooks/_data/archive/_data.zip
BLIMPConfig (fname: /Users/scottberry/.config/blimp/blimp.ini)
EXPERIMENT_DIR: /Users/scottberry/source/blimp
BASE_DATA_DIR: /Users/scottberry/source/blimp
data_config/exampledata: /Users/scottberry/source/blimp/notebooks/ExampleData_constants.py



In [2]:
from blimp.image import BLImage 
cyc01 = BLImage(example_data_path / '_data' / 'operetta_cls_multiplex' / 'cycle_01' / 'r05c03f15-fk1fl1-mip.ome.tiff')

The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


In [4]:
cyc01

<BLImage [Reader: OmeTiffReader, dtype: uint16, Image-is-in-Memory: False]>

In [ ]:
cyc01.x


In [ ]:
bl = cyc01.shift_yx(1,1)
bl.get_image_data('YX',Z=0,T=0,C=0)

In [ ]:
cyc01.shift_yx

In [ ]:
cyc01.get_image_data('YX',Z=0,T=0,C=0)

In [ ]:
from blimp.preprocessing.registration import TransformationParameters
cyc01.transformation_parameters = TransformationParameters(transformation_mode='rigid')
print(cyc01.transformation_parameters)

In [71]:
import dask.array as da
def convert_array_dtype(arr, dtype, round_floats=False):

    # check requested type is allowed
    allowed_types = np.sctypes['int'] + np.sctypes['uint'] + np.sctypes['float']
    if not dtype in allowed_types:
        raise TypeError("``dtype`` = {dtype} is either not recognised or not allowed")

    # convert input to a numpy array
    if isinstance(arr, da.core.Array):
        arr = np.ndarray(arr)
    if not isinstance(arr,np.ndarray):
        raise TypeError("``arr`` is not a ``numpy.ndarray``")

    old_dtype = arr.dtype
    new_dtype = dtype
    new = np.copy(arr)
    
    if old_dtype == new_dtype:
        pass

    elif old_dtype in np.sctypes['uint']:
        # convert from uint
        if np.can_cast(old_dtype, new_dtype):
            new = new.astype(new_dtype)
        else:
            raise TypeError(f"Cannot cast {old_dtype} to {new_dtype}")

    elif old_dtype in np.sctypes['int']:
        # convert from int
        if np.can_cast(old_dtype, new_dtype):
            new = new.astype(new_dtype)
        else:
            raise TypeError(f"Cannot cast {old_dtype} to {new_dtype}")

    elif old_dtype in np.sctypes['float']:
        # convert from float
        if not round_floats:
            if np.can_cast(old_dtype, new_dtype):
                new = new.astype(new_dtype)
            else:
                raise TypeError(f"Cannot cast {old_dtype} to {new_dtype}")
        else:
            # round to the nearest integer and represent using
            # the minimum scalar data type
            new = np.rint(new)
            min_dtype_based_on_rounded_values = np.promote_types(
                np.min_scalar_type(round(np.max(new))),
                np.min_scalar_type(round(np.min(new)))
            )
            new = new.astype(min_dtype_based_on_rounded_values)
            if np.can_cast(new.dtype, new_dtype):
                new = new.astype(new_dtype)
            else:
                raise RuntimeError(
                    f"Tried to convert array of ``float`` to an ``int`` or ``uint`` dtype" 
                    + f" but cannot convert to requested dtype ({new_dtype}) because the"
                    + f" minimum data type necessary to hold the values is ({new.dtype}) ")
    return new


In [73]:
arr = (np.random.random((10,10))*(2**17 - 1)).astype(np.float32)
convert_array_dtype(arr,np.uint16,round_floats=True)

RuntimeError: Tried to convert ``float`` to ``int`` or ``uint`` but cannot convert to requested dtype (<class 'numpy.uint16'>) because the minimum data type necessary to hold the values is (uint32) 

In [61]:
2**16 -1


65535